In [24]:
import urllib.request
from tqdm import tqdm_notebook as tqdm
from pymystem3 import Mystem
import pandas as pd
from urllib.parse import urlparse
import re
import os
import html
import time
from urllib.parse import quote                                                                                                                                                                

In [25]:
link_title = re.compile(r'<\/div>\r\n<a href=\".+?\"><div class=\"title\">.+?<\/div><\/a>', re.DOTALL)
page_link = re.compile(r'<\/div>\r\n<a href=\"(.+?)\"><div class=\"title\">.+?<\/div><\/a>', re.DOTALL)
page_title = re.compile(r'<h1>([^<>]+?)<\/h1>', re.DOTALL)
topic_dict = {'proishestviya': 'Происшествия', 'obschestvo': 'Общество', 'politika': 'Политика',
             'sport': 'Спорт', 'est_tema': 'Есть тема', 'kultura': 'Культура', 'pisatelstkaya': 'Писательская'} 
text_re = re.compile('<div class=\"news\-one\-content\">(.+?)<\/div>\r\n')
page_date = re.compile(r'<span class=\"date\">(.+?)<\/span>', re.DOTALL)

In [26]:
def clean(text):
    regTag = re.compile('<.*?>', re.DOTALL)
    regSpace = re.compile('\s{2,}', re.DOTALL)
    clean_text = regTag.sub(' ', text)
    clean_text = regSpace.sub(' ', clean_text)
    clean_text = html.unescape(clean_text)
    return clean_text

In [27]:
def make_date(date):
    months = {' января ': '.01.', ' февраля ': '.02.' ,' марта ': '.03.', ' апреля ': '.04.', ' мая ': '.05.', ' июня ': '.06.',
            ' июля ': '.07.', ' августа ': '.08.', ' сентября ': '.09.', ' октября ': '.10.', ' ноября ': '.11.', ' декабря ': '.12.'}
    for month, number in months.items():
        date = date.replace(month, number)
    return date

In [28]:
def make_files(text, title, date, url, topic):
    text1 = text
    year = re.findall(r'[0-9]{4,4}', date)[0]
    day = re.findall(r'[0-9]{2,2}', date)[0]
    month = re.findall(r'\.([0-9]{2,2})\.', date)[0]
    directory_plain = 'газета/plain/'+year+'/'+month
    directory_xml = 'газета/mystem-xml/'+year+'/'+month
    directory_mystem = 'газета/mystem-plain/'+year+'/'+month
    plain_path = directory_plain+'/'+quote(title)+'.txt'
    xml_path = directory_xml+'/'+quote(title)+'.xml'
    mystem_path = directory_mystem+'/'+quote(title)+'.txt'
    if not os.path.exists(directory_plain):
        os.makedirs(directory_plain)
    if not os.path.exists(directory_xml):
        os.makedirs(directory_xml)
    if not os.path.exists(directory_mystem):
           os.makedirs(directory_mystem)
    f = open(plain_path, 'w', encoding = 'utf-8')
    f.write(text)
    f.close()
    os.system('/Users/danya/project/mystem -c -i -l --eng-gr --format xml '+plain_path+' '+xml_path)
    os.system('/Users/danya/project/mystem -c -i -l --eng-gr '+plain_path+' '+mystem_path)
    f = open(plain_path, 'w', encoding = 'utf-8')
    f.write('@au None\n')
    f.write('@ti %s\n' % title)
    f.write('@da %s\n' % date)
    f.write('@topic %s\n' % topic)
    f.write('@url %s\n' % url)
    f.write(text)
    f.close()
    print ('Создан файл:'+' '+plain_path)
    return plain_path,year

In [29]:
def create_table(metatable, paths, headers, dates, topics, urls, years):
    metatable['path'] = pd.Series(paths).values
    metatable['author'] = 'None'
    metatable['header'] = pd.Series(headers).values
    metatable['created'] = pd.Series(dates).values
    metatable['sphere'] = 'публицистика'
    metatable['topic'] = pd.Series(topics).values
    metatable['style'] = 'нейтральный'
    metatable['audience_age'] = 'н-возраст'
    metatable['audience_level'] = 'н-уровень'
    metatable['audience_size'] = 'городская'
    metatable['source'] = pd.Series(urls)
    metatable['publ_year'] = pd.Series(years)
    metatable['medium'] = 'газета'
    metatable['country'] = 'Россия'
    metatable['region'] = 'Красноярский край'
    metatable['language'] = 'ru'
    return metatable

In [30]:
def download_pages():
    column_list = ['path', 'author', 'header', 'created', 'sphere', 'topic', 'style', 'audience_age',
                'audience_level', 'audience_size', 'source', 'publ_year', 'medium', 'country', 'region',
                'language']
    metatable = pd.DataFrame(columns = column_list)
    paths, headers, dates, topics, urls, years = [], [], [], [], [], []
    all_text = []
    i = 0
    topic_dict = {'proishestviya': 'Происшествия', 'obschestvo': 'Общество', 'politika': 'Политика',
             'sport': 'Спорт', 'est_tema': 'Есть тема', 'kultura': 'Культура', 'pisatelstkaya': 'Писательская'}
    while len((' '.join(all_text)).split()) < 200000:
        try:
            hub_url = 'http://www.gig26.ru/statii/p-'+str(i)+'.html' 
            hub =  urllib.request.urlopen(hub_url).read().decode('utf-8')
            links = re.findall(page_link, hub)
            for link in links:
                try:
                    url = 'http://www.gig26.ru'+link
                    print(url)
                    page = urllib.request.urlopen(url).read().decode('utf-8')
                    title = re.findall(page_title, page)[0]
                    print(title)
                    date = re.findall(page_date, page)[0]
                    date = make_date(date)
                    for top in topic_dict.keys():
                        if top in url:
                            topic = topic_dict[top]
                    print(topic)
                    print(date)
                    text = clean(re.findall(text_re, page)[0])
                    makefile = make_files(text, title, date, url, topic)
                    all_text.append(text)
                    print (len((' '.join(all_text)).split()))
                    paths.append(makefile[0])
                    headers.append(title)
                    dates.append(date)
                    topics.append(topic)
                    urls.append(url)
                    years.append(makefile[1])
                    time.sleep(1)
                except:
                    continue
        except:
            continue
        i+=1
    metatable = create_table(metatable, paths, headers, dates, topics, urls, years)
    return metatable

In [31]:
metatable = download_pages()

http://www.gig26.ru/statii/sport/nid-13688.html
Красиво, круто и атмосферно
Спорт
24.10.2018
Создан файл: газета/plain/2018/10/%D0%9A%D1%80%D0%B0%D1%81%D0%B8%D0%B2%D0%BE%2C%20%D0%BA%D1%80%D1%83%D1%82%D0%BE%20%D0%B8%20%D0%B0%D1%82%D0%BC%D0%BE%D1%81%D1%84%D0%B5%D1%80%D0%BD%D0%BE.txt
1153
http://www.gig26.ru/spetsproekty/istoriya_goroda/nid-13685.html
Александр Удовиченко: «Мы шли к одной цели»
Спорт
23.10.2018
Создан файл: газета/plain/2018/10/%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80%20%D0%A3%D0%B4%D0%BE%D0%B2%D0%B8%D1%87%D0%B5%D0%BD%D0%BA%D0%BE%3A%20%C2%AB%D0%9C%D1%8B%20%D1%88%D0%BB%D0%B8%20%D0%BA%20%D0%BE%D0%B4%D0%BD%D0%BE%D0%B9%20%D1%86%D0%B5%D0%BB%D0%B8%C2%BB.txt
1716
http://www.gig26.ru/statii/obschestvo/nid-13684.html
Не панацея, но шанс
Общество
23.10.2018
Создан файл: газета/plain/2018/10/%D0%9D%D0%B5%20%D0%BF%D0%B0%D0%BD%D0%B0%D1%86%D0%B5%D1%8F%2C%20%D0%BD%D0%BE%20%D1%88%D0%B0%D0%BD%D1%81.txt
3013
http://www.gig26.ru/statii/obschestvo/nid-13676.html
Найти человека


Шахерезада и танкист
Писательская
3.09.2018
Создан файл: газета/plain/2018/09/%D0%A8%D0%B0%D1%85%D0%B5%D1%80%D0%B5%D0%B7%D0%B0%D0%B4%D0%B0%20%D0%B8%20%D1%82%D0%B0%D0%BD%D0%BA%D0%B8%D1%81%D1%82.txt
34833
http://www.gig26.ru/spetsproekty/istoriya_goroda/nid-13432.html
Галстуки и ботинки шили в Доме быта
Писательская
3.09.2018
Создан файл: газета/plain/2018/09/%D0%93%D0%B0%D0%BB%D1%81%D1%82%D1%83%D0%BA%D0%B8%20%D0%B8%20%D0%B1%D0%BE%D1%82%D0%B8%D0%BD%D0%BA%D0%B8%20%D1%88%D0%B8%D0%BB%D0%B8%20%D0%B2%20%D0%94%D0%BE%D0%BC%D0%B5%20%D0%B1%D1%8B%D1%82%D0%B0.txt
35884
http://www.gig26.ru/spetsproekty/istoriya_goroda/nid-13434.html
Академии МЧС исполнилось 10 лет
Писательская
1.09.2018
Создан файл: газета/plain/2018/09/%D0%90%D0%BA%D0%B0%D0%B4%D0%B5%D0%BC%D0%B8%D0%B8%20%D0%9C%D0%A7%D0%A1%20%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D0%BD%D0%B8%D0%BB%D0%BE%D1%81%D1%8C%2010%20%D0%BB%D0%B5%D1%82.txt
37757
http://www.gig26.ru/statii/sport/nid-13418.html
Сражение у чистой Шуши
Спорт
30.08.2018
Создан файл: газета/p

Дорасти до Билла Гейтса
Есть тема
9.07.2018
Создан файл: газета/plain/2018/07/%D0%94%D0%BE%D1%80%D0%B0%D1%81%D1%82%D0%B8%20%D0%B4%D0%BE%20%D0%91%D0%B8%D0%BB%D0%BB%D0%B0%20%D0%93%D0%B5%D0%B9%D1%82%D1%81%D0%B0.txt
68714
http://www.gig26.ru/spetsproekty/istoriya_goroda/nid-13141.html
Лагерь с 45-летней историей
Есть тема
9.07.2018
Создан файл: газета/plain/2018/07/%D0%9B%D0%B0%D0%B3%D0%B5%D1%80%D1%8C%20%D1%81%2045-%D0%BB%D0%B5%D1%82%D0%BD%D0%B5%D0%B9%20%D0%B8%D1%81%D1%82%D0%BE%D1%80%D0%B8%D0%B5%D0%B9.txt
69961
http://www.gig26.ru/statii/obschestvo/nid-13181.html
Все тот же почтальон
Общество
7.07.2018
Создан файл: газета/plain/2018/07/%D0%92%D1%81%D0%B5%20%D1%82%D0%BE%D1%82%20%D0%B6%D0%B5%20%D0%BF%D0%BE%D1%87%D1%82%D0%B0%D0%BB%D1%8C%D0%BE%D0%BD.txt
71070
http://www.gig26.ru/spetsproekty/istoriya_goroda/nid-13152.html
Шиверские перекаты
Общество
6.07.2018
Создан файл: газета/plain/2018/07/%D0%A8%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%BA%D0%B8%D0%B5%20%D0%BF%D0%B5%D1%80%D0%B5%D0%BA%D0%B0%D1%82%D1

Готовы к водичке из ковшика?
Есть тема
24.04.2018
Создан файл: газета/plain/2018/04/%D0%93%D0%BE%D1%82%D0%BE%D0%B2%D1%8B%20%D0%BA%20%D0%B2%D0%BE%D0%B4%D0%B8%D1%87%D0%BA%D0%B5%20%D0%B8%D0%B7%20%D0%BA%D0%BE%D0%B2%D1%88%D0%B8%D0%BA%D0%B0%3F.txt
98579
http://www.gig26.ru/statii/politika/nid-12711.html
Гурий Двирный: «Мне говорят, хватит жить в розовых очках. А я живу»
Политика
20.04.2018
http://www.gig26.ru/spetsproekty/zheleznogortsy_vy_horoshie/nid-12693.html
Доброта и внимание бесценны
Политика
18.04.2018
Создан файл: газета/plain/2018/04/%D0%94%D0%BE%D0%B1%D1%80%D0%BE%D1%82%D0%B0%20%D0%B8%20%D0%B2%D0%BD%D0%B8%D0%BC%D0%B0%D0%BD%D0%B8%D0%B5%20%D0%B1%D0%B5%D1%81%D1%86%D0%B5%D0%BD%D0%BD%D1%8B.txt
98900
http://www.gig26.ru/statii/est_tema/nid-12680.html
Может, нанять сто китайцев?
Есть тема
16.04.2018
Создан файл: газета/plain/2018/04/%D0%9C%D0%BE%D0%B6%D0%B5%D1%82%2C%20%D0%BD%D0%B0%D0%BD%D1%8F%D1%82%D1%8C%20%D1%81%D1%82%D0%BE%20%D0%BA%D0%B8%D1%82%D0%B0%D0%B9%D1%86%D0%B5%D0%B2%3F.txt
99668


Пауза или отмена?
Общество
22.01.2018
Создан файл: газета/plain/2018/01/%D0%9F%D0%B0%D1%83%D0%B7%D0%B0%20%D0%B8%D0%BB%D0%B8%20%D0%BE%D1%82%D0%BC%D0%B5%D0%BD%D0%B0%3F.txt
118330
http://www.gig26.ru/statii/est_tema/nid-12243.html
Для кого поликлиника бережливая
Есть тема
21.01.2018
Создан файл: газета/plain/2018/01/%D0%94%D0%BB%D1%8F%20%D0%BA%D0%BE%D0%B3%D0%BE%20%D0%BF%D0%BE%D0%BB%D0%B8%D0%BA%D0%BB%D0%B8%D0%BD%D0%B8%D0%BA%D0%B0%20%D0%B1%D0%B5%D1%80%D0%B5%D0%B6%D0%BB%D0%B8%D0%B2%D0%B0%D1%8F.txt
119706
http://www.gig26.ru/spetsproekty/istoriya_goroda/nid-12227.html
Борис Беллер: лидер от бога
Есть тема
19.01.2018
Создан файл: газета/plain/2018/01/%D0%91%D0%BE%D1%80%D0%B8%D1%81%20%D0%91%D0%B5%D0%BB%D0%BB%D0%B5%D1%80%3A%20%D0%BB%D0%B8%D0%B4%D0%B5%D1%80%20%D0%BE%D1%82%20%D0%B1%D0%BE%D0%B3%D0%B0.txt
120304
http://www.gig26.ru/statii/proishestviya/nid-12226.html
На линии огня
Происшествия
19.01.2018
Создан файл: газета/plain/2018/01/%D0%9D%D0%B0%20%D0%BB%D0%B8%D0%BD%D0%B8%D0%B8%20%D0%BE%D0%B3%D

http://www.gig26.ru/statii/obschestvo/nid-11819.html
Брат на брата
Общество
1.11.2017
Создан файл: газета/plain/2017/11/%D0%91%D1%80%D0%B0%D1%82%20%D0%BD%D0%B0%20%D0%B1%D1%80%D0%B0%D1%82%D0%B0.txt
142453
http://www.gig26.ru/statii/obschestvo/nid-11766.html
Петр Гаврилов: «У меня самая интересная работа в мире»
Общество
30.10.2017
http://www.gig26.ru/statii/politika/nid-11768.html
Алексей Сергейкин: «Хочется создать что-то новое»
Политика
29.10.2017
http://www.gig26.ru/statii/obschestvo/nid-11762.html
Проруха на старика
Общество
27.10.2017
Создан файл: газета/plain/2017/10/%D0%9F%D1%80%D0%BE%D1%80%D1%83%D1%85%D0%B0%20%D0%BD%D0%B0%20%D1%81%D1%82%D0%B0%D1%80%D0%B8%D0%BA%D0%B0.txt
143457
http://www.gig26.ru/statii/sport/nid-11755.html
Бой с тенью
Спорт
26.10.2017
Создан файл: газета/plain/2017/10/%D0%91%D0%BE%D0%B9%20%D1%81%20%D1%82%D0%B5%D0%BD%D1%8C%D1%8E.txt
144971
http://www.gig26.ru/statii/kultura/nid-11754.html
Валентина Попова: «Современного человека в музей заманить непросто»
Культу

http://www.gig26.ru/spetsproekty/zheleznogortsy_vy_horoshie/nid-11366.html
Вот увидела Бабкину, и умирать можно!
Спорт
17.08.2017
Создан файл: газета/plain/2017/08/%D0%92%D0%BE%D1%82%20%D1%83%D0%B2%D0%B8%D0%B4%D0%B5%D0%BB%D0%B0%20%D0%91%D0%B0%D0%B1%D0%BA%D0%B8%D0%BD%D1%83%2C%20%D0%B8%20%D1%83%D0%BC%D0%B8%D1%80%D0%B0%D1%82%D1%8C%20%D0%BC%D0%BE%D0%B6%D0%BD%D0%BE%21.txt
169457
http://www.gig26.ru/spetsproekty/zheleznogortsy_vy_horoshie/nid-11362.html
Проверяй, но доверяй!
Спорт
17.08.2017
Создан файл: газета/plain/2017/08/%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D1%8F%D0%B9%2C%20%D0%BD%D0%BE%20%D0%B4%D0%BE%D0%B2%D0%B5%D1%80%D1%8F%D0%B9%21.txt
169746
http://www.gig26.ru/statii/est_tema/nid-11332.html
Десятиклассники в законе
Есть тема
11.08.2017
Создан файл: газета/plain/2017/08/%D0%94%D0%B5%D1%81%D1%8F%D1%82%D0%B8%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%BD%D0%B8%D0%BA%D0%B8%20%D0%B2%20%D0%B7%D0%B0%D0%BA%D0%BE%D0%BD%D0%B5.txt
171039
http://www.gig26.ru/statii/obschestvo/nid-11328.html
Игорь Колотупов

187967
http://www.gig26.ru/statii/obschestvo/nid-11060.html
Железногорск резко стареет
Общество
1.07.2017
Создан файл: газета/plain/2017/07/%D0%96%D0%B5%D0%BB%D0%B5%D0%B7%D0%BD%D0%BE%D0%B3%D0%BE%D1%80%D1%81%D0%BA%20%D1%80%D0%B5%D0%B7%D0%BA%D0%BE%20%D1%81%D1%82%D0%B0%D1%80%D0%B5%D0%B5%D1%82.txt
188464
http://www.gig26.ru/statii/est_tema/nid-11058.html
Валерий ГОЛОВКИН: «Не просто говорить «халва», а делать»
Есть тема
30.06.2017
http://www.gig26.ru/statii/sport/nid-11067.html
Играли все, а виноват один
Спорт
29.06.2017
Создан файл: газета/plain/2017/06/%D0%98%D0%B3%D1%80%D0%B0%D0%BB%D0%B8%20%D0%B2%D1%81%D0%B5%2C%20%D0%B0%20%D0%B2%D0%B8%D0%BD%D0%BE%D0%B2%D0%B0%D1%82%20%D0%BE%D0%B4%D0%B8%D0%BD.txt
189132
http://www.gig26.ru/statii/pisatelstkaya/nid-11034.html
Как один зеленогорец Железногорск полюбил
Писательская
27.06.2017
Создан файл: газета/plain/2017/06/%D0%9A%D0%B0%D0%BA%20%D0%BE%D0%B4%D0%B8%D0%BD%20%D0%B7%D0%B5%D0%BB%D0%B5%D0%BD%D0%BE%D0%B3%D0%BE%D1%80%D0%B5%D1%86%20%D0%96%D0%B5%D0%B

In [33]:
metatable.to_csv('metatable.csv', sep='\t', encoding = 'utf-8', index=False)